# Langfuse 提示词管理性能测试

## 测试目标和背景
本笔记本对 Langfuse 提示词管理系统进行性能基准测试，通过测量在无缓存模式下（cache_ttl_seconds=0）连续执行 1,000 次提示词检索和编译操作的延迟时间。

## 关于缓存的重要说明
在实际应用中，这种延迟通常不会成为问题，因为提示词会在 SDK 客户端进行缓存。更多关于缓存机制的信息，请参考 [Langfuse 提示词管理文档](https://langfuse.com/docs/prompt-management/features/caching)。

## 测试结果解读
测试结果包含了网络延迟，因此绝对数值可能会因地理位置和服务器负载而有所不同。建议使用直方图和统计摘要来比较相对性能改进，例如不同 SDK 版本或缓存设置之间的差异。

## 前置条件
测试需要在已认证的项目中设置一个名为 `perf-test` 的提示词模板。

## 适用人群
本教程适合大模型技术初学者，帮助理解提示词管理系统的性能特征和优化方法。

In [ ]:
# 安装 Langfuse Python SDK
# Langfuse 是一个开源的大语言模型应用可观测性平台
# 提供提示词管理、追踪、评估等功能
%pip install langfuse

In [ ]:
import os

# 配置 Langfuse 连接参数
# 这些密钥可以从项目设置页面获取：https://cloud.langfuse.com

# 公钥：用于客户端身份验证，相对安全可以暴露
os.environ["LANGFUSE_PUBLIC_KEY"] = ""

# 私钥：用于服务端身份验证，需要严格保密
os.environ["LANGFUSE_SECRET_KEY"] = ""

# 服务器地址配置
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"  # 🇪🇺 欧洲区域服务器
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com"  # 🇺🇸 美国区域服务器

# 注意：在生产环境中，建议使用 .env 文件或其他安全方式管理这些敏感信息

In [ ]:
# 导入必要的库
import time          # 用于时间测量和延迟控制
import pandas as pd  # 用于数据分析和统计计算
import matplotlib.pyplot as plt  # 用于数据可视化
from tqdm.auto import tqdm       # 用于显示进度条

from langfuse import Langfuse    # Langfuse 主要客户端类

# 从环境变量初始化 Langfuse 客户端
# 这会自动读取之前设置的 LANGFUSE_PUBLIC_KEY、LANGFUSE_SECRET_KEY 和 LANGFUSE_HOST
langfuse = Langfuse()

# 验证客户端是否正确初始化
# 如果认证失败，会抛出断言错误，提示检查环境变量配置
assert langfuse.auth_check(), "Langfuse 客户端初始化失败 - 请检查环境变量配置是否正确"

In [ ]:
# 性能测试配置参数
N_RUNS = 1_000  # 测试运行次数：1000次，足够获得统计学意义的结果
prompt_name = "perf-test"  # 提示词模板名称：需要在 Langfuse 项目中预先创建

# 存储每次操作的执行时间
durations = []

# 执行性能基准测试
for _ in tqdm(range(N_RUNS), desc="性能测试进行中"):
    # 记录开始时间（使用高精度计时器）
    start = time.perf_counter()
    
    # 获取提示词模板（禁用缓存以测试真实网络延迟）
    # cache_ttl_seconds=0 表示不使用缓存，每次都从服务器获取
    prompt = langfuse.get_prompt(prompt_name, cache_ttl_seconds=0)
    
    # 编译提示词（包含服务端处理时间）
    # 使用最小输入进行编译，模拟实际使用场景
    prompt.compile(input="test")
    
    # 计算并记录本次操作的总耗时
    durations.append(time.perf_counter() - start)
    
    # 添加短暂延迟，避免对服务器造成过大压力
    time.sleep(0.05)  # 50毫秒延迟

# 将时间数据转换为 pandas Series 便于后续分析
durations_series = pd.Series(durations, name="执行时间(秒)")

In [ ]:
# 计算详细的统计信息
# percentiles 参数指定要计算的百分位数
# 25%、50%（中位数）、75%、99% 分位数有助于理解数据分布
stats = durations_series.describe(percentiles=[0.25, 0.5, 0.75, 0.99])

# 显示统计结果
# count: 样本数量
# mean: 平均值
# std: 标准差（衡量数据离散程度）
# min/max: 最小值/最大值
# 25%/50%/75%/99%: 对应百分位数的值
stats

## 历史性能测试结果参考

以下是之前的性能测试结果，可以作为对比参考：

```
count    1000.000000  # 测试样本数：1000次
mean        0.039335 sec  # 平均响应时间：约39.3毫秒
std         0.014172 sec  # 标准差：约14.2毫秒（表示数据相对稳定）
min         0.032702 sec  # 最快响应时间：约32.7毫秒
25%         0.035387 sec  # 25%的请求在35.4毫秒内完成
50%         0.037030 sec  # 50%的请求在37.0毫秒内完成（中位数）
75%         0.041111 sec  # 75%的请求在41.1毫秒内完成
99%         0.068914 sec  # 99%的请求在68.9毫秒内完成
max         0.409609 sec  # 最慢响应时间：约409.6毫秒（可能是网络异常）
```

### 结果解读
- **平均响应时间约40毫秒**：对于大多数应用场景来说是可接受的
- **99%的请求在70毫秒内完成**：说明系统性能相对稳定
- **最大值较高**：可能由网络波动或服务器负载引起，属于正常现象

In [ ]:
# 创建性能数据的直方图可视化
plt.figure(figsize=(8,4))  # 设置图表大小：宽8英寸，高4英寸

# 绘制直方图
# bins=30 表示将数据分为30个区间，可以清晰显示数据分布
plt.hist(durations_series, bins=30, alpha=0.7, color='skyblue', edgecolor='black')

# 设置图表标签和标题
plt.xlabel("执行时间 (秒)")  # X轴标签
plt.ylabel("频次")         # Y轴标签
plt.title("Langfuse 提示词管理性能测试 - 响应时间分布")  # 图表标题

# 添加网格线，便于读取数值
plt.grid(True, alpha=0.3)

# 显示图表
plt.show()

# 直方图说明：
# - X轴：响应时间（秒）
# - Y轴：该时间范围内的请求数量
# - 图形形状：通常呈现正态分布或右偏分布
# - 峰值位置：表示最常见的响应时间
# - 长尾：表示少数异常慢的请求

## 历史测试结果图表参考

以下是之前性能测试的可视化结果，展示了响应时间的分布情况：

![性能测试图表](https://langfuse.com/images/docs/prompt-performance-chart.png)

### 图表解读要点
1. **分布形状**：大部分响应时间集中在较小的值附近
2. **峰值位置**：显示最常见的响应时间范围
3. **长尾现象**：少数请求的响应时间较长，这是网络应用的正常现象
4. **性能稳定性**：整体分布相对集中，说明系统性能稳定

### 实际应用建议
- **生产环境**：建议启用缓存（cache_ttl_seconds > 0）以获得更好的性能
- **监控指标**：关注99%分位数，确保绝大多数用户获得良好体验
- **优化策略**：可以通过CDN、地理位置优化等方式进一步提升性能